In [1]:
# -*- coding: utf-8 -*-

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
%matplotlib inline

import cv2

from keras.models import Model, load_model, model_from_yaml
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, Input, merge, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

from sklearn.metrics import jaccard_similarity_score
from sklearn.model_selection import train_test_split, ShuffleSplit

from glob import glob

Using TensorFlow backend.


In [2]:
def get_trained_model(model_file="u-net-model.yaml", weight_path="weights/unet_best.hdf5"):
    with open(model_file) as mf:
        model = model_from_yaml(mf.read())
    model.load_weights(weight_path)
    return model

In [3]:
trained_model = get_trained_model()
data_paths = []
for row_no in range(43):
    for col_no in range(57):
        data_paths.append("pieces/image_{:03}_{:03}.png".format(row_no, col_no))
data = []
for path in data_paths:
    data.append(cv2.imread(path))
pred_y = trained_model.predict(
    np.stack(data, axis=0),
    verbose=1
)

2451/2451 [==============================] - 911s   


In [4]:
n_rows, n_cols = 43, 57
result = np.zeros((n_rows*160, n_cols*160, 1), pred_y.dtype)
for i in range(n_rows):
    for j in range(n_cols):
        result[i*160:i*160+160, j*160:j*160+160] = pred_y[i*n_cols+j]
plt.imsave("predict0904.tif", result.reshape(result.shape[0], result.shape[1]))

In [5]:
pred_y.shape

(2451, 160, 160, 1)

In [7]:
np.save("predict0904", result.reshape(result.shape[0], result.shape[1]))